# TRANSFORMACIÓN DE DATOS

## IMPORTAR PAQUETES

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import os
import re
#    import unidecode
#    import nltk
#    nltk.download('stopwords')
#    from nltk.corpus import stopwords
import seaborn as sns
sns.set_style('darkgrid')
import warnings
warnings.filterwarnings('ignore')


#    from sklearn.preprocessing import OneHotEncoder
#    from sklearn.preprocessing import OrdinalEncoder
#    from sklearn.preprocessing import LabelEncoder
#    from sklearn.preprocessing import KBinsDiscretizer
#    from sklearn.preprocessing import KBinsDiscretizer
#    from sklearn.preprocessing import Binarizer
#    from sklearn.preprocessing import PowerTransformer
#    from sklearn.preprocessing import QuantileTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
#    from sklearn.preprocessing import MinMaxScaler
#    from sklearn.preprocessing import StandardScaler
#    from sklearn.preprocessing import RobustScaler
#    from sklearn.preprocessing import MaxAbsScaler
#    from imblearn.under_sampling import RandomUnderSampler
#    from imblearn.over_sampling import RandomOverSampler



#Automcompletar rápido
%config IPCompleter.greedy=True

## IMPORTAR LOS DATOS

1.- Sustituir la ruta del proyecto.

In [2]:
ruta_proyecto = 'C:/Users/Usuario/miniconda3/SAI Dia FBS/'

2.- Nombrar los ficheros de datos.

In [3]:
nombre = 'df_resultado_eda.pickle'

3.- Cargar los datos.

In [4]:
df = pd.read_pickle(ruta_proyecto + '/02_Datos/03_Trabajo/' + nombre)

4.- Separar la target.

In [5]:
target = df[['cno']].copy().reset_index(drop=True)

In [6]:
first_n_cno = df['cno'].value_counts().reset_index().iloc[:10, :]
first_n_cno

,index,cno
0,3811,398
1,3820,192
2,2461,183
3,2713,178
4,5120,154
5,2719,136
6,2121,125
7,3510,122
8,5110,107
9,9210,92


In [7]:
# Obtenemos 

df['cno'].value_counts().reset_index().rename( columns={'cno':'Conteo', 'index': 'cno'}).iloc[:10, :]

,cno,Conteo
0,3811,398
1,3820,192
2,2461,183
3,2713,178
4,5120,154
5,2719,136
6,2121,125
7,3510,122
8,5110,107
9,9210,92


In [8]:
CNO_elegidos = ['3811', '3820', '2461', '5120'] #'2713',

df = df[df['cno'].isin(CNO_elegidos)].reset_index()

In [9]:
df['cno'].value_counts().reset_index().iloc[:10, :]

,index,cno
0,3811,398
1,3820,192
2,2461,183
3,5120,154


### PREPROCESAMIENTO

#### LIMPIEZA DE TEXTOS

##### TITULO OFERTA

In [10]:
df.iloc[50, :]

index                                                           877
id                                                       1763338814
cno                                                            5120
titulo_oferta                     Camarero/a-encargado/a-cocinero/a
detalle_oferta    Si eres una persona dinámica, detallista, poli...
empresa_oferta                                    PZ.ZURRAQUE 00006
categoria         Hostelería/turismo -\r\nCamareros\r\nCamareros...
Name: 50, dtype: object

In [11]:
df['titulo_oferta'][95]

'Ayudante de camarero'

Se puede observar en este ejemplo que el texto del campo 'titulo_oferta' no se limita a nombrar puesto del trabajo ofertado. Por tanto realizaremos de una limpieza de carácteres, acentos, stopwords y revisaremos el listado del conjunto de las palabras para las oferta e eliminar la innecesarias. 

In [12]:
provincias_lower = []

provincias=["Álava", "Albacete", "Alicante", "Almería", "Asturias", "Ávila", "Badajoz", "Barcelona", "Burgos", "Cáceres", "Cádiz", "Cantabria", "Castellón", "Ciudad Real", "Córdoba", "Cuenca", "Gerona", "Granada", "Guadalajara", "Guipúzcoa", "Huelva", "Huesca", "Islas Baleares", "Jaén", "La Coruña", "La Rioja", "Las Palmas", "León", "Lérida", "Lugo", "Madrid", "Málaga", "Murcia", "Navarra", "Orense", "Palencia", "Pontevedra", "Salamanca", "Santa Cruz de Tenerife", "Segovia", "Sevilla", "Soria", "Tarragona", "Teruel", "Toledo", "Valencia", "Valladolid", "Vizcaya", "Zamora", "Zaragoza"]

for i in provincias: 
    x = i.lower()
#    x = unidecode.unidecode(x)
    provincias_lower.append(x)

In [13]:
provincias_lower

['álava',
 'albacete',
 'alicante',
 'almería',
 'asturias',
 'ávila',
 'badajoz',
 'barcelona',
 'burgos',
 'cáceres',
 'cádiz',
 'cantabria',
 'castellón',
 'ciudad real',
 'córdoba',
 'cuenca',
 'gerona',
 'granada',
 'guadalajara',
 'guipúzcoa',
 'huelva',
 'huesca',
 'islas baleares',
 'jaén',
 'la coruña',
 'la rioja',
 'las palmas',
 'león',
 'lérida',
 'lugo',
 'madrid',
 'málaga',
 'murcia',
 'navarra',
 'orense',
 'palencia',
 'pontevedra',
 'salamanca',
 'santa cruz de tenerife',
 'segovia',
 'sevilla',
 'soria',
 'tarragona',
 'teruel',
 'toledo',
 'valencia',
 'valladolid',
 'vizcaya',
 'zamora',
 'zaragoza']

In [14]:
stop_words_spanish = ['de',
'la',
'que',
'el',
'en',
'y',
'a',
'los',
'del',
'se',
'las',
'por',
'un',
'para',
'con',
'no',
'una',
'su',
'al',
'lo',
'como',
'más',
'pero',
'sus',
'le',
'ya',
'o',
'este',
'sí',
'porque',
'esta',
'entre',
'cuando',
'muy',
'sin',
'sobre',
'también',
'me',
'hasta',
'hay',
'donde',
'quien',
'desde',
'todo',
'nos',
'durante',
'todos',
'uno',
'les',
'ni',
'contra',
'otros',
'ese',
'eso',
'ante',
'ellos',
'e',
'esto',
'mí',
'antes',
'algunos',
'qué',
'unos',
'yo',
'otro',
'otras',
'otra',
'él',
'tanto',
'esa',
'estos',
'mucho',
'quienes',
'nada',
'muchos',
'cual',
'poco',
'ella',
'estar',
'estas',
'algunas',
'algo',
'nosotros',
'mi',
'mis',
'tú',
'te',
'ti',
'tu',
'tus',
'ellas',
'nosotras',
'vosostros',
'vosostras',
'os',
'mío',
'mía',
'míos',
'mías',
'tuyo',
'tuya',
'tuyos',
'tuyas',
'suyo',
'suya',
'suyos',
'suyas',
'nuestro',
'nuestra',
'nuestros',
'nuestras',
'vuestro',
'vuestra',
'vuestros',
'vuestras',
'esos',
'esas',
'estoy',
'estás',
'está',
'estamos',
'estáis',
'están',
'esté',
'estés',
'estemos',
'estéis',
'estén',
'estaré',
'estarás',
'estará',
'estaremos',
'estaréis',
'estarán',
'estaría',
'estarías',
'estaríamos',
'estaríais',
'estarían',
'estaba',
'estabas',
'estábamos',
'estabais',
'estaban',
'estuve',
'estuviste',
'estuvo',
'estuvimos',
'estuvisteis',
'estuvieron',
'estuviera',
'estuvieras',
'estuviéramos',
'estuvierais',
'estuvieran',
'estuviese',
'estuvieses',
'estuviésemos',
'estuvieseis',
'estuviesen',
'estando',
'estado',
'estada',
'estados',
'estadas',
'estad',
'he',
'has',
'ha',
'hemos',
'habéis',
'han',
'haya',
'hayas',
'hayamos',
'hayáis',
'hayan',
'habré',
'habrás',
'habrá',
'habremos',
'habréis',
'habrán',
'habría',
'habrías',
'habríamos',
'habríais',
'habrían',
'había',
'habías',
'habíamos',
'habíais',
'habían',
'hube',
'hubiste',
'hubo',
'hubimos',
'hubisteis',
'hubieron',
'hubiera',
'hubieras',
'hubiéramos',
'hubierais',
'hubieran',
'hubiese',
'hubieses',
'hubiésemos',
'hubieseis',
'hubiesen',
'habiendo',
'habido',
'habida',
'habidos',
'habidas',
'soy',
'eres',
'es',
'somos',
'sois',
'son',
'sea',
'seas',
'seamos',
'seáis',
'sean',
'seré',
'serás',
'será',
'seremos',
'seréis',
'serán',
'sería',
'serías',
'seríamos',
'seríais',
'serían',
'era',
'eras',
'éramos',
'erais',
'eran',
'fui',
'fuiste',
'fue',
'fuimos',
'fuisteis',
'fueron',
'fuera',
'fueras',
'fuéramos',
'fuerais',
'fueran',
'fuese',
'fueses',
'fuésemos',
'fueseis',
'fuesen',
'sintiendo',
'sentido',
'sentida',
'sentidos',
'sentidas',
'siente',
'sentid',
'tengo',
'tienes',
'tiene',
'tenemos',
'tenéis',
'tienen',
'tenga',
'tengas',
'tengamos',
'tengáis',
'tengan',
'tendré',
'tendrás',
'tendrá',
'tendremos',
'tendréis',
'tendrán',
'tendría',
'tendrías',
'tendríamos',
'tendríais',
'tendrían',
'tenía',
'tenías',
'teníamos',
'teníais',
'tenían',
'tuve',
'tuviste',
'tuvo',
'tuvimos',
'tuvisteis',
'tuvieron',
'tuviera',
'tuvieras',
'tuviéramos',
'tuvierais',
'tuvieran',
'tuviese',
'tuvieses',
'tuviésemos',
'tuvieseis',
'tuviesen',
'teniendo',
'tenido',
'tenida',
'tenidos',
'tenidas',
'tened']

In [15]:
# Creamos la función de limpoieza de textos


def clean(text):   
    # Quitar las expresiones entre corchetes 
    text = re.sub(r'\[.+?\]', ' ', str(text))
    # Quita las aclaraciones entre paréntesis
    text = re.sub(r'\(.+?\)', ' ', text)
    # Quitar acentos
    # text = unidecode.unidecode(text)
    # Quitar aquello que no sean palabras o cosa que se le parezca
    text = re.sub(r'\W', ' ', text)
    # Quitar espacios extra en caso de haber
    text = re.sub(r'\s+', ' ', text)
    # Quitar un conjunto de carácteres que son intelegibles y se repiten a lo largo de los campos
    text = re.sub('\r\n', ' ', text)
    # Pasa el texto a minúsulas
    text = text.lower()
    # Quita palabras 
    text = re.sub(r'se busca', ' ', text)
    # Quita palabras 
    text = re.sub(r'se necesita', ' ', text)
    # Retira stopwords
    text = text.split()
    text = [ word for word in text  if word not in stop_words_spanish] #stopwords.words('spanish')]
    text = ' '.join(text)
    return text



In [16]:
# Realizamos la limpieza

df['titulo_oferta'] = df['titulo_oferta'].apply(lambda x: clean(x))


In [17]:
# Realizamos el primer vistazo a un regitro de TITULO OFERTA

df['titulo_oferta'].reset_index().iloc[90, 1]



'administrador sistemas basados azure'

In [18]:
# Creamos un listado de todas la palabras utilizadas para la oferta en una lista
words_list = []
for i in range(0, len(df)):
    text = df['titulo_oferta'].reset_index().iloc[i, 1]
    for word in text.split():
        words_list.append(word)
words_list

# Creamos el corpus de las palabras del Titulo Oferta
# oferta_corpus = " ".join(words_list)



['camarero',
 'experiencia',
 'camarera',
 'camarero',
 'camarera',
 'camarero',
 'ingles',
 'barra',
 'sala',
 'camarero',
 'trabajar',
 'fines',
 'semana',
 'festivos',
 'programador',
 'full',
 'stack',
 'programador',
 'backend',
 'c',
 'sharp',
 'sql',
 'archivex',
 'clinical',
 'camarero',
 'desarrollador',
 'back',
 'end',
 'java',
 'spring',
 'precisa',
 'camarero',
 'camarero',
 'camarero',
 'camarera',
 'técnico',
 'soporte',
 'sistemas',
 'informáticos',
 'técnico',
 'sistemas',
 'informáticos',
 'tic',
 'camarero',
 'camarero',
 'precisan',
 'cocineros',
 'camareros',
 'restaurante',
 'gastronómico',
 'camarero',
 'restaurante',
 'consultor',
 'técnico',
 'software',
 'hcm',
 'camarero',
 'sala',
 'terraza',
 'supplier',
 'development',
 'engineer',
 'lead',
 'ayudante',
 'camarero',
 'técnico',
 'calidad',
 'camarero',
 'ingeniero',
 'industrialización',
 'piezas',
 'vehículo',
 'camarer',
 'camarero',
 'camarero',
 'jefa',
 'azafatas',
 'experiencia',
 'yates',
 '40m',
 '

In [19]:
# Creamos un diccionario de las palabras y su frecuencia.

def listaPalabrasDicFrec(listaPalabras):
    frecuenciaPalab = [listaPalabras.count(p) for p in listaPalabras]
    return dict(list(zip(listaPalabras,frecuenciaPalab)))

# Ordemanos el diccionario por la frecuencia de la palabra
def ordenaDicFrec(dicfrec):
    aux = [(dicfrec[key], key) for key in dicfrec]
    aux.sort()
    aux.reverse()
    return aux 

In [20]:
ordenaDicFrec(listaPalabrasDicFrec(words_list))

[(204, 'madrid'),
 (186, 'programador'),
 (144, 'ingeniero'),
 (113, 'remoto'),
 (109, '100'),
 (106, 'java'),
 (104, 'barcelona'),
 (93, 'técnico'),
 (93, 'camarero'),
 (70, 'proyectos'),
 (65, 'desarrollador'),
 (60, 'junior'),
 (52, 'developer'),
 (50, 'net'),
 (49, 'senior'),
 (47, 'sistemas'),
 (42, 'instalaciones'),
 (34, 'valencia'),
 (33, 'software'),
 (27, 'camareros'),
 (23, 'soporte'),
 (22, 'empleo'),
 (22, 'as'),
 (21, 'primer'),
 (20, 'engineer'),
 (19, 'responsable'),
 (17, 'fullstack'),
 (17, 'c'),
 (17, 'bizkaia'),
 (16, 'programadores'),
 (15, 'php'),
 (15, 'lead'),
 (15, 'full'),
 (15, 'ciberseguridad'),
 (15, 'ayudante'),
 (15, 'administrador'),
 (14, 'stack'),
 (14, 'frontend'),
 (14, 'backend'),
 (14, 'angular'),
 (13, 'valladolid'),
 (13, 'jefe'),
 (13, 'industrial'),
 (13, 'front'),
 (13, 'camarera'),
 (13, 'analista'),
 (12, 'sql'),
 (12, 'manager'),
 (12, 'general'),
 (12, 'desarrollo'),
 (12, 'consultor'),
 (11, 'sevilla'),
 (11, 'sala'),
 (11, 'end'),
 (10, 

In [21]:
def remove_provincia(text):
    text = text.split()
    text = [ word for word in text if word not in provincias_lower]
    text = ' '.join(text)
    return text

In [22]:
# Realizamos la limpieza

df['titulo_oferta'] = df['titulo_oferta'].apply(lambda x: remove_provincia(x))

In [23]:
# Creamos un listado de todas la palabras utilizadas para la oferta en una lista
words_list = []
for i in range(0, len(df)):
    text = df['titulo_oferta'].reset_index().iloc[i, 1]
    for word in text.split():
        words_list.append(word)
words_list

# Ordenamos de nuevo por frecuencia de las palabras. # madrid ya no aparece. 
ordenaDicFrec(listaPalabrasDicFrec(words_list))

[(186, 'programador'),
 (144, 'ingeniero'),
 (113, 'remoto'),
 (109, '100'),
 (106, 'java'),
 (93, 'técnico'),
 (93, 'camarero'),
 (70, 'proyectos'),
 (65, 'desarrollador'),
 (60, 'junior'),
 (52, 'developer'),
 (50, 'net'),
 (49, 'senior'),
 (47, 'sistemas'),
 (42, 'instalaciones'),
 (33, 'software'),
 (27, 'camareros'),
 (23, 'soporte'),
 (22, 'empleo'),
 (22, 'as'),
 (21, 'primer'),
 (20, 'engineer'),
 (19, 'responsable'),
 (17, 'fullstack'),
 (17, 'c'),
 (17, 'bizkaia'),
 (16, 'programadores'),
 (15, 'php'),
 (15, 'lead'),
 (15, 'full'),
 (15, 'ciberseguridad'),
 (15, 'ayudante'),
 (15, 'administrador'),
 (14, 'stack'),
 (14, 'frontend'),
 (14, 'backend'),
 (14, 'angular'),
 (13, 'jefe'),
 (13, 'industrial'),
 (13, 'front'),
 (13, 'camarera'),
 (13, 'analista'),
 (12, 'sql'),
 (12, 'manager'),
 (12, 'general'),
 (12, 'desarrollo'),
 (12, 'consultor'),
 (11, 'sala'),
 (11, 'end'),
 (10, 'seguridad'),
 (10, 'react'),
 (10, 'javascript'),
 (9, 'usuarios'),
 (9, 'restaurante'),
 (9, 'q

##### DETALLE OFERTA

In [24]:
df['detalle_oferta']

0      Camarero con experiencia, jornada completa, tu...
1      Tomar comandas. Llevar rango. Marchar segundos...
2      Se necesita persona responsable para puesto de...
3      Se necesita camarero con ingles para barra y s...
4      Se necesita persona para cubrir puesto de cama...
                             ...                        
922                                         Programación
923    SERVIR BARRA Y COMEDOR\r\nDatos de contacto:\r...
924                     Sólo con experiencia de camamer@
925    DESDE TEMPORING SELECCIONAMOS CAMAREROS/AS PAR...
926    SE PRECISA PROGRAMADOR/A INFORMÁTICO/A JUNIOR ...
Name: detalle_oferta, Length: 927, dtype: object

In [25]:
df['detalle_oferta'][66]

'Ayudante de camarera a jornada completa de lunes a sabado, muy valorado seriedad, sinceridad e ingles.'

In [26]:
df['detalle_oferta'] = df['detalle_oferta'].apply(lambda x: clean(x))

In [27]:
# Creamos el corpus de todas la palabras utilizadas para la oferta en una lista
words_list = []
for i in range(0, len(df)):
    text = df['detalle_oferta'].reset_index().iloc[i, 1]
    for word in text.split():
        words_list.append(word)
        
ordenaDicFrec(listaPalabrasDicFrec(words_list))

[(891, 'experiencia'),
 (594, 'desarrollo'),
 (556, 'proyectos'),
 (530, 'empresa'),
 (496, 'equipo'),
 (394, 'java'),
 (355, 'años'),
 (326, 'conocimientos'),
 (316, 'programador'),
 (303, 'proyecto'),
 (303, 'madrid'),
 (301, 'sistemas'),
 (296, 'sector'),
 (260, 'clientes'),
 (256, 'gestión'),
 (252, 'parte'),
 (250, 'trabajar'),
 (249, 'trabajo'),
 (244, 'cliente'),
 (243, 'técnico'),
 (235, 'nivel'),
 (231, 'software'),
 (229, 'servicios'),
 (219, 'requisitos'),
 (216, 'ingeniería'),
 (216, 'buscamos'),
 (211, 'net'),
 (192, 'soluciones'),
 (191, 'departamento'),
 (186, 'si'),
 (182, 'personas'),
 (181, 'soporte'),
 (180, 'profesional'),
 (179, 'aplicaciones'),
 (178, 'funciones'),
 (177, 'persona'),
 (176, 'instalaciones'),
 (169, 'remoto'),
 (169, '100'),
 (168, 'tecnologías'),
 (163, 'perfil'),
 (160, 'tecnología'),
 (159, 'puesto'),
 (159, '2'),
 (158, 'ingeniero'),
 (155, 'sql'),
 (154, 'buscando'),
 (154, 'actualmente'),
 (153, 'datos'),
 (152, 'barcelona'),
 (149, 'importan

In [28]:
df['detalle_oferta'][66]

'ayudante camarera jornada completa lunes sabado valorado seriedad sinceridad ingles'

In [29]:
df['detalle_oferta'].isna().sum()

0

##### CNO

In [30]:
df['cno'][66]

'5120'

In [31]:
df['cno'].value_counts()

3811    398
3820    192
2461    183
5120    154
Name: cno, dtype: int64

In [32]:
df['cno'].value_counts().reset_index()[0:9]

,index,cno
0,3811,398
1,3820,192
2,2461,183
3,5120,154


##### CATEGORIA


In [33]:
df['categoria'][0]

'Hostelería/turismo -\r\nCamareros\r\nCamareros Asalariados\r\nMARINELA LIVIA CUREAN\r\n15/03/2022'

In [34]:
df['categoria'].isna().sum()

0

In [35]:
df['categoria'] = df['categoria'].fillna("0")

In [36]:
df['categoria'] = df['categoria'].apply(lambda x: clean(x))

In [37]:
df['categoria'][0]

'hostelería turismo camareros camareros asalariados marinela livia curean 15 03 2022'

In [38]:
# Creamos el corpus de todas la palabras utilizadas para la oferta en una lista
words_list = []
for i in range(0, len(df)):
    text = df['categoria'].reset_index().iloc[i, 1]
    for word in text.split():
        words_list.append(word)

#oferta_corpus = " ".join(words_list)

ordenaDicFrec(listaPalabrasDicFrec(words_list))

[(590, 'informáticos'),
 (581, 'técnicos'),
 (570, '2022'),
 (537, 'telecomunicaciones'),
 (534, 'informática'),
 (501, 'programadores'),
 (481, 'sistemas'),
 (398, 'operaciones'),
 (325, '02'),
 (309, 'analistas'),
 (307, '2021'),
 (284, 'camareros'),
 (204, '03'),
 (186, 'producción'),
 (183, 'ingenieros'),
 (183, 'industriales'),
 (154, 'asalariados'),
 (151, 'ciencias'),
 (151, 'calidad'),
 (149, 'proempleoingenieros'),
 (149, 'cogiti'),
 (147, 'ingeniería'),
 (143, 'turismo'),
 (143, 'hostelería'),
 (133, '12'),
 (132, 'ingenierías'),
 (131, '04'),
 (122, '01'),
 (87, '08'),
 (86, '09'),
 (80, 'seguridad'),
 (80, 'redes'),
 (72, 'asistencia'),
 (66, 'técnica'),
 (66, 'microinformática'),
 (66, '10'),
 (63, '11'),
 (61, '07'),
 (51, '06'),
 (42, 'primer'),
 (42, 'empleo'),
 (40, '15'),
 (39, '14'),
 (38, '16'),
 (36, 's'),
 (36, '05'),
 (33, '20'),
 (32, 'l'),
 (31, '28'),
 (31, '13'),
 (29, '2020'),
 (29, '18'),
 (28, '22'),
 (28, '17'),
 (27, '25'),
 (27, '24'),
 (27, '23'),
 (22

In [39]:
df.iloc[0,:]

index                                                             6
id                                                       1764985510
cno                                                            5120
titulo_oferta                                  camarero experiencia
detalle_oferta    camarero experiencia jornada completa turnos r...
empresa_oferta                                MARINELA LIVIA CUREAN
categoria         hostelería turismo camareros camareros asalari...
Name: 0, dtype: object

## GUARDAR DATASET TRAS TRANSFORMACIÓN DE DATOS

En formato pickle para no perder las modificaciones de metadatos.

In [40]:
#Definir los nombres del archivo
ruta = ruta_proyecto + '/02_Datos/03_Trabajo/' + 'df_transf.pickle'

In [41]:
#Guardar los archivos
df.to_pickle(ruta)